# Image Analysis Pipeline

This markdown document describes an image analysis pipeline implemented in Python using OpenCV and NumPy libraries.

## Code Overview

The provided Python script implements several functions for analyzing images within a specified folder. Here's a summary of the functions:

1. `load_images_from_folder`: Load images from a specified folder path.
2. `save_image`: Save an image to the specified output path with a transformation name added to the filename.
3. `resize_image`: Resize an image to a target size.
4. `calculate_damp_area`: Calculate the relative damp area between two images, ignoring patches originating from the specimen boundary.
5. `overlay_damp_areas`: Overlay thresholded damp areas onto the original image.
6. `visualize_damp_area_heatmap`: Visualize damp areas on the image using heatmaps.
7. `main`: Main function to execute the image analysis pipeline.

## Usage

To use this script, you need to have Python installed along with the OpenCV and NumPy libraries.

You can run the script by executing it from the command line or an integrated development environment (IDE).

```bash
python image_analysis.py


In [9]:
import cv2
import numpy as np
import os

def load_images_from_folder(folder_path):
    """
    Load images from a specified folder path.

    Parameters:
    - folder_path (str): The folder path containing the images.

    Returns:
    - images (list): A list of loaded images.
    """
    images = []
    for filename in sorted(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    return images

def save_image(image, output_path, transformation_name):
    """
    Save an image to the specified output path with the transformation name added to the filename.

    Parameters:
    - image (numpy.ndarray): The image to save.
    - output_path (str): The file path where the image will be saved.
    - transformation_name (str): The name of the transformation applied to the image.
    """
    cv2.imwrite(os.path.join(output_path, transformation_name + '.jpg'), image)

def resize_image(image, target_size):
    """
    Resize an image to the target size.

    Parameters:
    - image (numpy.ndarray): The input image.
    - target_size (tuple): The target size (width, height) for resizing.

    Returns:
    - resized_image (numpy.ndarray): The resized image.
    """
    return cv2.resize(image, target_size)

def calculate_damp_area(ref_image, final_image, boundary_threshold=20):
    """
    Calculate the relative damp area between two images, ignoring patches originating from the specimen boundary.

    Parameters:
    - ref_image (numpy.ndarray): The reference image.
    - final_image (numpy.ndarray): The final image.
    - boundary_threshold (int): Threshold distance from the specimen boundary to ignore contours.

    Returns:
    - damp_area (float): The relative damp area.
    - thresh (numpy.ndarray): The binary image after thresholding, resized to match the dimensions of the reference image.
    - contours (list): A list of contours representing damp areas, excluding those originating from the specimen boundary.
    """
    if ref_image.shape[:2] != final_image.shape[:2]:
        final_image = resize_image(final_image, (ref_image.shape[1], ref_image.shape[0]))

    ref_gray = cv2.cvtColor(ref_image, cv2.COLOR_BGR2GRAY)
    final_gray = cv2.cvtColor(final_image, cv2.COLOR_BGR2GRAY)

    diff_image = cv2.absdiff(ref_gray, final_gray)
    _, thresh = cv2.threshold(diff_image, 50, 255, cv2.THRESH_BINARY)

    # Resize the thresholded image to match the dimensions of the reference image
    thresh = resize_image(thresh, (ref_image.shape[1], ref_image.shape[0]))

    kernel = np.ones((5, 5), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    boundary_contours, _ = cv2.findContours(cv2.Canny(ref_gray, 30, 100), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boundary = max(boundary_contours, key=cv2.contourArea)

    filtered_contours = []
    for contour in contours:
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            if cv2.pointPolygonTest(boundary, (cX, cY), measureDist=True) > boundary_threshold:
                filtered_contours.append(contour)

    damp_area = np.sum(opening) / np.sum(ref_gray)

    return damp_area, thresh, filtered_contours, ref_gray, diff_image, opening

def overlay_damp_areas(image, thresh):
    """
    Overlay thresholded damp areas onto the original image.

    Parameters:
    - image (numpy.ndarray): The original image.
    - thresh (numpy.ndarray): The binary image after thresholding.

    Returns:
    - overlay_image (numpy.ndarray): The original image with damp areas overlaid.
    """
    thresh_resized = cv2.resize(thresh, (image.shape[1], image.shape[0]))
    
    overlay_image = image.copy()
    overlay_image[thresh_resized != 0] = [0, 0, 255]  # Set damp areas to red (BGR color)
    return overlay_image

def visualize_damp_area_heatmap(thresh):
    """
    Visualize damp areas on the image using heatmaps.

    Parameters:
    - thresh (numpy.ndarray): The binary image after thresholding.

    Returns:
    - heatmap (numpy.ndarray): The heatmap of damp areas.
    """
    heatmap = cv2.applyColorMap(thresh, cv2.COLORMAP_JET)
    return heatmap

def main():
    folder_paths = [
        '/home/chemweno/Desktop/ProjectFinal/brickspictures',
        '/home/chemweno/Desktop/ProjectFinal/Sample 1 - Nakuru',
        '/home/chemweno/Desktop/ProjectFinal/Sample 2 - Mogotio',
        '/home/chemweno/Desktop/ProjectFinal/Sample 4 - Lessos',
        '/home/chemweno/Desktop/ProjectFinal/bush stones'
    ]

    for folder_path in folder_paths:
        results_folder = os.path.join(folder_path, 'results')
        if not os.path.exists(results_folder):
            os.makedirs(results_folder)

        images = load_images_from_folder(folder_path)

        # Use the first image as the reference
        ref_image = images[0]

        # Open or create a text file to store the results
        results_file_path = os.path.join(results_folder, 'results.txt')
        with open(results_file_path, 'w') as results_file:
            for i, image in enumerate(images):
                if i == 0:  # Skip the first image, which is the reference
                    continue

                # Check if dimensions match
                if ref_image.shape[:2] != image.shape[:2]:
                    image = resize_image(image, (ref_image.shape[1], ref_image.shape[0]))

                damp_area, _, _, _, _, _ = calculate_damp_area(ref_image, image)

                results_file.write(f"Relative damp area for image {i+1}: {damp_area}\n")

                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                save_image(gray_image, results_folder, f'image_{i+1}_gray')

                # Ensure both images have the same dimensions for absolute difference calculation
                ref_gray = cv2.cvtColor(ref_image, cv2.COLOR_BGR2GRAY)
                diff_image = cv2.absdiff(ref_gray, gray_image)
                save_image(diff_image, results_folder, f'image_{i+1}_diff')

                _, thresh = cv2.threshold(diff_image, 50, 255, cv2.THRESH_BINARY)
                save_image(thresh, results_folder, f'image_{i+1}_thresh')

                kernel = np.ones((5, 5), np.uint8)
                opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
                save_image(opening, results_folder, f'image_{i+1}_opening')

                heatmap = visualize_damp_area_heatmap(opening)
                save_image(heatmap, results_folder, f'image_{i+1}_heatmap')

                overlay_image = overlay_damp_areas(image, opening)
                save_image(overlay_image, results_folder, f'image_{i+1}_overlay')

    print("Results saved to the respective folders")

if __name__ == "__main__":
    main()


Results saved to the respective folders
